In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from utils.utils import *

In [1]:
#CLUSTERS = ["cosmos08", "cosmos09", "cosmos14", "cosmos15"]
CLUSTERS = ["cosmos08", "cosmos09", "cosmos14", "cosmos15"]

FN_TMP = "../data/{CLUSTER}/raw_views_filters_2021-12-15-augmented.ss.csv"
#DEDUPFN = FN.replace(".ss.csv", "-dedup.ss.csv")
#AUGMENTED_FN = FN.replace(".ss.csv", "-augmented.ss.csv")

INP_FIELD = "NormInputHashCode"
FILTER_FIELD = "FilterExpr"
DEDUP_FIELDS = [INP_FIELD, FILTER_FIELD]

STATCOLS = ["like_ops", "discrete_ops", "cont_ops"]

In [2]:
alldfs = []
for cluster in CLUSTERS:
    fn = FN_TMP.format(CLUSTER=cluster)
    df = pd.read_csv(fn, sep="\t")
    df["cluster"] = cluster
    alldfs.append(df)

NameError: name 'pd' is not defined

In [5]:
df = pd.concat(alldfs)

In [6]:
def pretty_print(df):
    return display( HTML( df.to_html(index=False).replace("\\n","<br>") ) )

# Exploring main classes

In [7]:
STATCOLS = ["discrete_ops", "cont_ops", "like_ops", "regex_ops"]
STATSTODROP = ["std", "min", "max", "25%", "50%","75%"]
curdrops = []

for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#cdf = df[["dataset","like_ops", "cont_ops", "discrete_ops", "in_ops"]]
#cdf = pd.concat([cdf,scopetmp])

pretty_print(df.groupby(["cluster"])[STATCOLS].describe().\
     reset_index().\
     drop(columns=curdrops).\
     rename(columns={"like_ops":"LIKE", "discrete_ops":"Discrete", "cont_ops":"Continuous",
                     "regex_ops":"Regex",
                    "mean":"Fraction"}).\
     round(3))

In [8]:
timedata = defaultdict(list)

for cluster in set(df["cluster"]):
    tmp1 = df[df.cluster == cluster]
    totaltime = tmp1["ExclusiveTime"].sum()
    timedata["cluster"].append(cluster)
    timedata["total"].append(totaltime)
    for col in STATCOLS:
        tmp2 = tmp1[tmp1[col] == 1]
        curtime = tmp2["ExclusiveTime"].sum()
        timedata[col].append(curtime / totaltime)

In [9]:
tdf = pd.DataFrame(timedata)
#tdf["total"] = int(tdf["total"] / 3600)
tdf["total"] = tdf.apply(lambda x: int(x["total"]/3600), axis=1)

In [10]:
import copy
#curdrops2 = copy.deepcopy(curdrops) + [('discrete_ops', 'count')]

curstatcols = ["total", "discrete_ops", "cont_ops", "like_ops", "regex_ops"]
pretty_print(tdf.groupby(["cluster"])[curstatcols].mean().\
     reset_index().\
     #drop(columns=curdrops2).\
     rename(columns={"like_ops":"LIKE", "discrete_ops":"Discrete", "cont_ops":"Continuous",
                     "regex_ops":"Regex",
                     "total": "Total (hours)",
                    "mean":"Fraction of time"}).\
     round(3))

cluster,Total (hours),Discrete,Continuous,LIKE,Regex
cosmos08,632905.0,0.301,0.413,0.127,0.007
cosmos09,151981.0,0.657,0.401,0.051,0.105
cosmos14,118810.0,0.736,0.175,0.485,0.001
cosmos15,164480.0,0.599,0.569,0.199,0.001


# Likes

In [11]:
dflikes = df[df.like_ops == 1]
dflikes = dflikes[dflikes.like_kind != "no%"]
dflikes = dflikes[dflikes.like_kind != "multi%"]

curdrops = []
STATCOLS = ["ExclusiveTime", "like_lens", "RowCount", "Selectivity"]
STATSTODROP = ["std", "min", "max", "99%"]
for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#curdrops.append(tuple(["QError","mean"]))
curdrops.append(tuple(["RowCount","mean"]))
curdrops.append(tuple(["Selectivity","mean"]))

dflikes.groupby(["like_kind", "cluster"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
     reset_index().\
     drop(columns=curdrops).\
     set_index(["like_kind", "cluster"]).\
     rename(columns={"like_lens":"Filter Length"}).\
     round(2)

ExclusiveTime                              Filter Length  \
                           count      mean      50%       90%          mean   
like_kind cluster                                                             
contains  cosmos08       10501.0  18126.74     6.54   4848.74         11.79   
          cosmos09       18415.0    552.03     0.00      8.04         11.88   
          cosmos14       91701.0   2160.70     0.02      3.18          9.42   
          cosmos15      118218.0    891.10     0.37     42.81          9.98   
ends      cosmos08         229.0  92035.09     1.04  77097.69          5.91   
          cosmos09         173.0  13050.25     3.86    743.91          7.42   
          cosmos14         315.0  12968.59  2656.75  11248.48          3.48   
          cosmos15         401.0   8781.92   366.80  28259.15          7.43   
starts    cosmos08        2849.0  26480.83     0.06   1405.84         16.14   
          cosmos09        3964.0   3889.68     2.32   5784.85          8.74   
          cosmos14        1266.0   4165.75    31.85   1001.52         15.56   
          cosmos15       14371.0    603.96     0.31    199.45         11.81   

                                  RowCount               Selectivity        
                     50%   90%         50%           90%         50%   90%  
like_kind cluster                                                           
contains  cosmos08  11.0  17.0   3164646.0  1.822244e+09        0.01  0.59  
          cosmos09  12.0  19.0         0.0  4.729859e+06        0.00  0.02  
          cosmos14   7.0  24.0         1.0  7.537870e+05        0.00  0.00  
          cosmos15   7.0  18.0     89014.0  6.281940e+06        0.02  0.31  
ends      cosmos08   6.0   6.0     57109.0  2.193271e+09        0.00  0.34  
          cosmos09   6.0  16.0    322969.0  4.403996e+08        0.00  0.79  
          cosmos14   2.0   6.0    253443.0  1.299988e+06        0.00  0.00  
          cosmos15   6.0   6.0  80239568.0  2.590911e+09        0.00  0.03  
starts    cosmos08   5.0  42.0     19984.0  3.574099e+08        0.00  0.40  
          cosmos09   8.0   9.0  20463134.0  3.188837e+08        0.00  0.92  
          cosmos14   2.0  22.0       119.5  2.901699e+08        0.00  0.24  
          cosmos15   4.0  34.0     86394.0  7.629605e+08        0.00  0.63

In [15]:
dflikes = df[df.like_ops == 1]
dflikes = dflikes[dflikes.like_kind != "no%"]
dflikes = dflikes[dflikes.like_kind != "multi%"]

curdrops = []
STATCOLS = ["ExclusiveTime", "like_lens", "RowCount", "Selectivity"]
STATSTODROP = ["std", "min", "max", "99%"]
for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

#curdrops.append(tuple(["QError","mean"]))
curdrops.append(tuple(["RowCount","mean"]))
curdrops.append(tuple(["Selectivity","mean"]))

dflikes.groupby(["like_dtype", "cluster"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
     reset_index().\
     drop(columns=curdrops).\
     set_index(["like_dtype", "cluster"]).\
     rename(columns={"like_lens":"Filter Length"}).\
     round(2)

ExclusiveTime                                  \
                            count       mean      50%         90%   
like_dtype cluster                                                  
extension  cosmos08          95.0  463131.24     3.55  1030939.51   
           cosmos09           7.0      21.26     3.01       57.53   
           cosmos14           4.0       0.01     0.00        0.02   
           cosmos15          96.0     198.29    16.21      410.07   
hex        cosmos14           1.0       0.11     0.11        0.11   
           cosmos15          93.0    5273.04  2492.60    18226.39   
num        cosmos08        1570.0    2452.72     0.05     2362.58   
           cosmos09          64.0     238.11     0.00        3.48   
           cosmos14        3124.0    1782.07    34.39      255.73   
           cosmos15         650.0    3654.03     0.61     3351.01   
path       cosmos08         419.0   29895.49   167.36     3763.65   
           cosmos09          55.0    1308.85     0.11      363.72   
           cosmos14         229.0     877.96   116.97      130.58   
           cosmos15        1119.0     837.51     0.00      458.47   
serial     cosmos08          50.0     576.82     3.83      339.40   
           cosmos09          37.0    6442.14     0.20     6297.73   
           cosmos14         280.0    4244.06     7.35     9310.62   
           cosmos15        3270.0      44.39     0.38        6.45   
short      cosmos08        1600.0   23241.93     8.56     7407.38   
           cosmos09        1479.0     956.96     0.72      966.62   
           cosmos14        1270.0    6936.71   743.51    32669.18   
           cosmos15        7926.0     521.69     0.03      229.32   
unknown    cosmos08        7574.0   20886.41     0.20     4445.01   
           cosmos09       20680.0    1027.59     0.00       15.58   
           cosmos14       78748.0    2427.10     0.02        0.70   
           cosmos15       97950.0     989.78     0.25        7.98   
url        cosmos08        2271.0   13687.73   817.88    41213.14   
           cosmos09         230.0   21088.93    19.77   110772.79   
           cosmos14        9626.0      62.54     0.00        0.25   
           cosmos15       21886.0     570.87    23.41     1101.69   

                    Filter Length                  RowCount                \
                             mean   50%    90%          50%           90%   
like_dtype cluster                                                          
extension  cosmos08         12.16   8.0   15.0    2511117.0  1.684665e+09   
           cosmos09          8.86   3.0   18.6    2694592.0  1.240169e+07   
           cosmos14          7.00   5.0   10.6          2.0  7.002600e+03   
           cosmos15          9.07   4.0   10.5      11005.5  2.472169e+06   
hex        cosmos14         10.00  10.0   10.0       7833.0  7.833000e+03   
           cosmos15          8.44  10.0   10.0    6000764.0  1.237933e+08   
num        cosmos08          4.18   4.0    6.0       1510.5  4.400573e+07   
           cosmos09          5.44   6.0    7.0          1.0  3.906160e+04   
           cosmos14          2.01   2.0    2.0      64618.0  1.866353e+06   
           cosmos15          5.48   3.0    9.0     134318.5  9.792497e+09   
path       cosmos08         19.09   8.0   56.0    3818142.0  7.223291e+08   
           cosmos09         19.11  21.0   30.0         62.0  1.587002e+06   
           cosmos14         29.00  19.0   49.2   48950300.0  5.025216e+07   
           cosmos15         47.76  55.0   63.2         58.0  1.516486e+06   
serial     cosmos08         33.62  36.0   36.0         54.0  6.567696e+06   
           cosmos09         55.08  36.0  221.0       9162.0  4.288522e+06   
           cosmos14         85.24  36.0  363.0        277.5  7.518300e+08   
           cosmos15         23.91  28.0   28.0    2802235.5  3.432072e+07   
short      cosmos08          1.13   1.0    2.0    5749270.0  2.663860e+08   
           cosmos09          0.89   1.0    2.0  

In [19]:
tmp = dflikes[dflikes.like_dtype == "extension"]
tmp = tmp[tmp.cluster == "cosmos08"]
tmp = tmp.sort_values(by="ExclusiveTime", ascending=False)
print(tmp["ExclusiveTime"].head(10))
tmp["FilterExpr"].head(10).values

824804    2.836637e+07
856559    5.285502e+06
57996     1.165789e+06
332432    1.147001e+06
508064    1.128335e+06
540980    1.120111e+06
403937    1.110833e+06
342970    1.104273e+06
712695    1.097732e+06
228146    1.063051e+06
Name: ExclusiveTime, dtype: float64


array(['{"isComparison":false,"expOperator":"And","attribute":{"name":"","dataType":""},"values":[],"children":[{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType":"String"},"values":["\\"h-----com.google.\\""],"children":[],"isSorted":false},{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType":"String"},"values":["\\"h-----com.msn.\\""],"children":[],"isSorted":false},{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType":"String"},"values":["\\"h-----com.youtube.\\""],"children":[],"isSorted":false},{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType":"String"},"values":["\\"h-----org.wikipedia.\\""],"children":[],"isSorted":false},{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType":"String"},"values":["\\"s-----com.facebook.\\""],"children":[],"isSorted":false},{"isComparison":true,"expOperator":"!=","attribute":{"name":"Domain:0","dataType

In [93]:
# pretty_print(dflikes.groupby(["like_kind"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
#      reset_index().\
#      sort_values(by=("Selectivity","count"), ascending=False).\
#      #drop(columns=curdrops).\
#      rename(columns={"like_lens":"Filter Length"}).\
#      round(2))

In [20]:
df.keys()

Index(['jobid', 'HashTagInput', 'OperatorName', 'EstCardinality',
       'InputCardinality', 'NormInputHashCode', 'RowCount', 'Selectivity',
       'ExclusiveTime', 'StageName', 'VertexCount', 'FilterExpr', 'QError',
       'num_ops', 'num_unique_ops', 'unique_filter_cols', 'like_ops',
       'like_lens', 'like_dtype', 'like_kind', 'like_casting', 'like_const',
       'discrete_ops', 'discrete_eqs', 'discrete_noneqs', 'discrete_type',
       'discrete_types_all', 'null_checks', 'cont_ops', 'cont_dates',
       'cont_others', 'cont_type', 'cont_dtype', 'cont_range', 'cont_cols',
       'types_all', 'complex_ops', 'complex_ops_num', 'udf_ops', 'in_ops',
       'regex_ops', 'equal_dates', 'num_discrete_consts', 'cluster'],
      dtype='object')

In [118]:
dfcont = df[df.cont_ops == 1]
STATCOLS = ["ExclusiveTime", "Selectivity", "QError", "VertexCount"]
curdrops = []
for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))
dfcont = dfcont[dfcont.cont_type != "other"]
dfcont = dfcont[dfcont.cont_type != "discont"]
        
curdrops.append(tuple(["QError","mean"]))
dfcont.groupby(["cont_type", "cluster"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
 reset_index().\
 set_index(["cont_type", "cluster"]).\
 rename(columns={"cont_type":"Continuous Op Type"}).\
 drop(columns=curdrops).round(2)

ExclusiveTime                        Selectivity        \
                           count     mean   50%     90%        mean   50%   
cont_type cluster                                                           
gt        cosmos08      133939.0   738.68  0.05   31.31        0.06  0.02   
          cosmos09       51939.0   894.34  0.44   39.50        0.27  0.00   
          cosmos14      202024.0    26.35  0.03    0.11        0.83  1.00   
          cosmos15      591306.0   116.90  0.00   10.51        0.17  0.01   
lt        cosmos08       61478.0  1486.40  0.03   36.69        0.34  0.06   
          cosmos09        8210.0   199.86  0.02   53.95        0.46  0.15   
          cosmos14        4097.0   281.39  0.00  125.95        0.12  0.00   
          cosmos15      215897.0    66.99  0.03   10.39        0.61  0.83   
range     cosmos08       42436.0   619.42  1.40  448.10        0.17  0.00   
          cosmos09      327771.0   344.00  2.10   46.48        0.02  0.00   
          cosmos14      196074.0    68.91  0.16    4.09        0.63  0.79   
          cosmos15      568591.0   325.87  0.09   56.62        0.17  0.01   

                               QError               VertexCount          \
                     90%          50%           90%        mean     50%   
cont_type cluster                                                         
gt        cosmos08  0.10        27.45  9.175900e+02      182.69    25.0   
          cosmos09  1.00        32.84  6.220861e+04      226.91    32.0   
          cosmos14  1.00         6.15  6.076000e+01      220.41   250.0   
          cosmos15  0.92        40.01  3.936710e+03      135.52     2.0   
lt        cosmos08  1.00         9.68  3.191500e+02      348.53    52.0   
          cosmos09  1.00        27.23  1.210135e+05      277.65     6.0   
          cosmos14  0.44  67684100.00  1.089530e+08      230.03   250.0   
          cosmos15  1.00         1.98  4.079000e+01      116.68    65.0   
range     cosmos08  1.00       161.43  3.130130e+03     1099.77  1000.0   
          cosmos09  0.01       240.22  3.023257e+05      267.02    43.0   
          cosmos14  1.00         4.91  1.870070e+06      150.37    36.0   
          cosmos15  1.00        51.88  6.553760e+06      191.34     6.0   

                            
                       90%  
cont_type cluster           
gt        cosmos08   250.0  
          cosmos09   500.0  
          cosmos14   250.0  
          cosmos15   192.0  
lt        cosmos08  1000.0  
          cosmos09   946.3  
          cosmos14   250.0  
          cosmos15   250.0  
range     cosmos08  2000.0  
          cosmos09  2000.0  
          cosmos14   250.0  
          cosmos15   250.0

In [102]:
STATCOLS = ["ExclusiveTime", "Selectivity", "QError", "VertexCount"]
curdrops = []
for si, stc in enumerate(STATCOLS):
    for sdrop in STATSTODROP:
        curdrops.append(tuple([stc, sdrop]))
    if si != 0:
        curdrops.append(tuple([stc,"count"]))

curdrops.append(tuple(["QError","mean"]))
pretty_print(dfcont.groupby(["cont_dtype"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
     reset_index().sort_values(by=("Selectivity","count"), ascending=False).\
     rename(columns={"cont_dtype":"Continuous Data \\n Type"}).\
     drop(columns=curdrops).round(2))

In [110]:
dfr = df[df.cont_range != 0.0]
dfr = dfr[dfr.cont_dtype == "date"]
dfr["cont_range_days"] = dfr.apply(lambda x: x["cont_range"] / 86400 ,axis=1)

#HTML
pretty_print(dfr.groupby(["cluster"])["cont_range_days"].describe(percentiles=[0.25, 0.40,
                            0.6, 0.75, 0.9, 0.99]).\
     reset_index().drop(columns=["std", "mean", "min", "max"]).sort_values(by="count", ascending=False).round(3))

cluster,count,25%,40%,50%,60%,75%,90%,99%
cosmos15,434098.0,14.239,28.000,30.000,30.000,30.042,92.000,1674.417
cosmos14,358568.0,10.667,10.700,10.736,10.753,27.000,57.042,554.042
cosmos09,327527.0,1.125,1.438,1.646,1.854,180.000,365.000,365.000
cosmos08,39948.0,0.042,0.042,0.417,1.000,41.042,1418.000,3652059.000


In [112]:
# #df2 = df[df["types_all"].count(",") <= 2]
# #df2
# types_all = set(df["types_all"].values)
# new_types_all = set()
# for t in types_all:
#     #t2 = t.replace("null,", "")
#     if "null" in t:
#         continue
#     if t.count(",") == 2:
#         new_types_all.add(t)

# df2 = df[df.types_all.isin(new_types_all)]
# # HTML(df2.groupby(["types_all"])["Selectivity"].describe(percentiles=[0.9, 0.99]).\
# #      reset_index().sort_values(by="count", ascending=False).round(2).to_html(index=False))

# STATCOLS = ["ExclusiveTime", "Selectivity", "QError", "VertexCount"]
# curdrops = []
# for si, stc in enumerate(STATCOLS):
#     for sdrop in STATSTODROP:
#         curdrops.append(tuple([stc, sdrop]))
#     if si != 0:
#         curdrops.append(tuple([stc,"count"]))
# curdrops.append(tuple(["QError","mean"]))

# pretty_print(df2.groupby(["types_all"])[STATCOLS].describe(percentiles=[0.9, 0.99]).\
#      reset_index().sort_values(by=("Selectivity","count"), ascending=False).\
#      rename(columns={"cont_type":"Continuous Operator \\n Type"}).\
#      drop(columns=curdrops).round(2))

In [117]:
discdf = df[df.discrete_ops == 1]
discdf.groupby(["cluster"])["num_discrete_consts"].describe(percentiles=[0.9, 0.99])

,count,mean,std,min,50%,90%,99%,max
cluster,,,,,,,,
cosmos08,626976.0,2.211710,2.670003,0.0,1.0,4.0,20.0,50.0
cosmos09,321889.0,1.178658,1.196281,0.0,1.0,1.0,4.0,50.0
cosmos14,3018006.0,7.701247,8.630881,0.0,4.0,20.0,39.0,50.0
cosmos15,1991941.0,1.485200,2.600382,0.0,1.0,2.0,10.0,50.0
